In [1]:
#%%writefile library_management_1.py
"""This module is used for managing a library. 
     There are many modules for managing your books_data."""
import pymysql as sql
from datetime import date, timedelta
from getpass import getpass
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from random import randint
class librarian:
    # Librarian class with and related methods 
    def __init__(self,l_id,password):
        self.l_id = l_id
        self.password = password
        self.db =  sql.connect(host='localhost', port=3306, user='root', password='',database = "library")
        self.cursor = self.db.cursor()
        
        
    def show_details_of_book(self):
       #show_details_of_book
        
        try:
            self.cursor.execute('select * from books_data')
            detail = self.cursor.fetchall()
            for i in detail:
                print(f"\nBook_Id  :  {i[0]}   &     Book_Name   :   {i[1]}      &     Book_Author   :   {i[2]}    &     Book_Published_Year   :    {i[3]}      &       Book_Category   :  {i[4]} " )
                print("__"*60)
                print()
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
            
    def users_who_have_to_submit_their_book_on_that_date(self):
        #Detail of the students whose today is last date of book  
        
        try:
            t = date.today().isoformat()
            self.cursor.execute(f"with temp as (select user_id,student_name,book_id,submission_date from user_book_data inner join user_data on user_book_data.user_id = user_data.id) select * from temp where submission_date = '{t}'")
            detail = self.cursor.fetchall()
            if len(detail):
                for i in detail:
                    print(f"\n User_Id :  {i[0]}  &    User_Name   :   {i[1]}     &     Book_Id   :   {i[2]}    &      Submission_Date    :   {i[3]}")
                    print("__"*60)
            else:
                print("\n\t\t\t\t\t************ No One **************")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
        
        
            
    def users_who_have_issued_the_books(self):
        #Details of the users who have issued the books 
        
        try:
            self.cursor.execute("with temp as (select * from user_book_data inner join user_data on user_book_data.user_id = user_data.id) select id,student_name,book_id,submission_date from temp;")
            new = self.cursor.fetchall()
            for i in new:
                print(f" \n USER ID :  {i[0]}   &   Student NAME :  {i[1]}   &    ID OF BOOKS   :   {i[2]}    &     SUBMISSION DATE  :   {i[3]}")
                print("__"*60)
                print()
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
            
    def send_email(self):
        #send email those student whose today is last date of book
        try:
            from_email = "hiteshnarwani111@gmail.com"
            password = getpass("\n PLEASE ENTER THE PASSWORD OF FROM EMAIL  :  ")
            t = date.today().isoformat()
            message = MIMEMultipart()
            message['From'] = "hiteshnarwani111@gmail.com"
            self.cursor.execute(f"with temp as (select id,user_email,submission_date from user_data inner join user_book_data on user_data.id = user_book_data.user_id) select user_email,submission_date from temp where submission_date = '{t}'")                 
            email = self.cursor.fetchall()
            host = "smtp.gmail.com"
            port = 465
            for i in email:
                message['To'] = i[0]
                message['Subject'] = "RAJASTHAN UNIVERSITY"
                msg = """Hello Dear,
                Today is your Last Date Of Book  
                Please Sumbit the Book
                      Thanks"""

                plain = MIMEText(msg, 'plain')
                message.attach(plain)


                with smtplib.SMTP_SSL(host, port) as server:
                    server.login(from_email, password)
                    server.sendmail(from_email, message['To'], message.as_string())
                    print(f"\n\t\t\t\t\t**************** MESSAGE HAVE BEEN SENT TO {message['To']} ****************")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
    def insert_book(self): 
        #New insert book in the library.
        try:
            b_i = int(input("\nPLEASE ENTER THE BOOK ID  :  "))
            b_n = input("\nPLEASE ENTER THE BOOK NAME  :  ").strip().title()
            b_a = input("\nPLEASE ENTER THE BOOK AUTHOR  :  ").strip().title()
            b_p = int(input("\n PLEASE ENTER THE BOOK PUBLISHED YEAR  :  "))
            b_c = input("\n PLEASE ENTER THE BOOK CATEGORY  :  ").strip().title()
            cmd = f"insert into books_data values({b_i},'{b_n}','{b_a}',{b_p},'{b_c}')"
            self.cursor.execute(cmd)
            self.db.commit()
            print("\n\t\t\t ************** BOOK INSERTED SUCCESSFULLY *****************")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
            
    def submit_book(self):
        try:
            u_i = int(input("\nPLEASE ENTER THE USER ID WHO WANNA SUBMIT THEIR BOOK  :  "))
            cmd = f"select * from user_book_data where user_id = {u_i}"
            self.cursor.execute(cmd)
            f = self.cursor.fetchall()
            if len(f)>0:
                l = []
                for i in f:
                    l.append(i[2])
                    print(f"\nUser_Id  :  {i[0]}\t&\t Student_Name  :  {i[1]}\t&\t Book_Id  :  {i[2]}\t&\t Book_Name  :  {i[3]}\t&\t Submission_Date  :  {i[4]}")
                    print("__"*60)
                b_i = int(input("\nPLEASE ENTER THE BOOK ID THAT YOU WANT TO SUMBIT  : "))
                if b_i in l:
                    r = f"delete from user_book_data where book_id = {b_i}"
                    self.cursor.execute(r)
                    self.db.commit()
                    print("\n\t\t\t\t\t****************** THANKS FOR SUBMIT ********************")
                else:
                    print("\n\t\t\t\t\tPLEASE ENTER THE VALID BOOK ID !!!!!!!!!!")
            else:
                print("\n\t\t\t\t\tUSER HAS NOT ISSUED ANY BOOK!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
class user_login:
    #User_login and Related Some methods 
    def __init__(self,user_name,password,id_):
        self.user_name = user_name
        self.password = password
        self.id_ = id_
        self.db =  sql.connect(host='localhost', port=3306, user='root', password='',database = "library")
        self.cursor = self.db.cursor()
        
        
    def get_book_by_author(self):
        #if you want to book issue by author_name 
        try:
            self.cursor.execute(f"select count({self.id_}) from user_book_data where user_id = {self.id_}")
            ide = self.cursor.fetchall()
            if ide[0][0]<3:
                self.cursor.execute("select book_author,book_name,book_id from books_data")
                authors = self.cursor.fetchall()
                print()
                for i in authors:
                    print(f"AUTHOR NAME : {i[0]}   &      BOOK NAME : {i[1]}")
                    print("__"*40)
                    print()
                author_name = input("\nENTER THE AUTHOR NAME WHICH YOU WANT TO BOOK :  ").strip().title()
                l = [i[0] for i in authors]
                if l.count(author_name)>1:
                    k = []
                    for j in authors:
                        if j[0] == author_name:
                            print(f"\n BOOK ID  :  {j[2]}.    BOOK_NAME   :    {j[1]}")
                            k.append(j[2])
                    b_i = int(input("\nPLEASE ENTER THE BOOK ID WHICH BOOK YOU WANT  : "))
                    if b_i in k:
                        self.cursor.execute("select book_id from user_book_data")
                        id__ = self.cursor.fetchall()
                        p = [i[0] for i in id__]
                        if b_i not in p:
                            self.cursor.execute(f"select book_name from books_data where book_id = '{b_i}'")
                            id1 = self.cursor.fetchall()
                            self.cursor.execute(f"select user_name from user_data where id = {self.id_}")
                            name = self.cursor.fetchall()
                            t = (date.today()+timedelta(days=30)).isoformat()
                            cmd = f"insert into user_book_data(user_id,student_name,book_id,book_name,Submission_date) values({self.id_},'{name[0][0]}',{b_i},'{id1[0][0]}','{t}')"
                            self.cursor.execute(cmd)
                            self.db.commit()    
                            print()
                            print("\n\t\t\t\******* BOOK ISSUED SUCCESSFULLY *********")
                        else:
                            print("\n\t\t\t\t\t THIS BOOK IS NOT PRESENT AT THIS TIME IN THE LIBRARY !!!!!!!!!!!!1")
                    else:
                        print("\n\t\t\t\t\t PLEASE ENTER THE VALID BOOK_ID !!!!!!!!!!!!!!")       

                elif l.count(author_name)==1:
                    for j in authors:
                        if j[0] == author_name:        
                            self.cursor.execute(f"select book_id from books_data where book_author = '{author_name}'")
                            b_i = self.cursor.fetchall()
                            B_I = b_i[0][0]
                            print(f"{B_I}.  {author_name}")
                            t = (date.today()+timedelta(days=30)).isoformat()
                            self.cursor.execute("select book_id from user_book_data")
                            id__ = self.cursor.fetchall()
                            self.cursor.execute(f"select book_name from books_data where book_id = '{B_I}'")
                            id1 = self.cursor.fetchall()
                            self.cursor.execute(f"select user_name from user_data where id = {self.id_}")
                            name = self.cursor.fetchall()
                            t = (date.today()+timedelta(days=30)).isoformat()
                            p = [i[0] for i in id__]
                            if B_I not in p:
                                cmd = f"insert into user_book_data values({self.id_},'{name[0][0]}',{B_I},'{id1[0][0]}','{t}')"
                                self.cursor.execute(cmd)
                                self.db.commit()
                                print(f"\n\t\t\t\t\t YOU HAVE ISSUED ( {j[1]} ) BOOK")
                                print()
                                print("\n \t\t\t\t********* BOOK ISSUED SUCCESSFULLY ********")
                                break
                            else:
                                print("\n\t\t\t\t\t THIS BOOK IS NOT PRESENT IN THE LIBRARY !!!!!!!!!!!!!")
                else:
                    print("\n\t\t\t\t\t THIS USER NAME BOOK IS NOT PRESENT IN THE LIBRARY !!!!!!!!!!!")
            else:
                print("\n\t\t\t\t\t ONLY MAXIMUM THREE BOOKS ARE ALLOWED !!!!!!!!!!!!!!")
                
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")

            
    def Get_book_by_category(self):
        #if you want to book issue by category
        try:
            self.cursor.execute(f"select count({self.id_}) from user_book_data where user_id = '{self.id_}'")
            ide = self.cursor.fetchall()
            if ide[0][0]<3:
                category = input("\n PLEASE ENTER THE BOOK CATEGORY  :  ").strip().lower().title()
                self.cursor.execute(f'select book_id,book_name, Book_Category from books_data where Book_category = "{category}"')
                cat = self.cursor.fetchall()
                for i in cat:
                    print()
                    print(f"Book_Id   :   {i[0]}     &     Book_Name   :    {i[1]}     &      Book_Category    :   {i[2]}")
                    print("__"*60)
                l = [i[2] for i in cat]
                if category in l:
                    b_i = int(input("\n PLEASE ENTER THE BOOK ID WHICH YOU WANT  :  "))
                    k = []
                    for u in cat:
                        k.append(u[0])
                    if b_i in k:
                        self.cursor.execute("select book_id from user_book_data")
                        id__ = self.cursor.fetchall()
                        p = [i[0] for i in id__]
                        if b_i not in p:
                            t = (date.today()+timedelta(days=30)).isoformat()
                            self.cursor.execute(f"select book_name from books_data where book_id = '{b_i}'")
                            id1 = self.cursor.fetchall()
                            self.cursor.execute(f"select user_name from user_data where id = {self.id_}")
                            name = self.cursor.fetchall()
                            cmd = f"insert into user_book_data values({self.id_},'{name[0][0]}',{b_i},'{id1[0][0]}','{t}')" 
                            self.cursor.execute(cmd)
                            self.db.commit()
                            print()
                            print("\n\t\t\t\t\t*********************** BOOK ISSUED SUCCESSFULLY *************************")
                        else:
                            print("\n\t\t\t\t\tTHIS BOOK IS NOT PRESENT IN THE LIBRARY !!!!!!!!!!!!!")
                    else:
                        print("\n\t\t\t\t\t PLEASE ENTER THE BOOK ID FROM THESE !!!!!!!!!!")
                else:
                    print("\n \t\t\t\t\t THIS CATEGORY TYPE BOOK IS NOT PRESENT IN THE LIBRARY !!!!!!!!!!!!!")
            else:
                print("\n \t\t\t\t\t ONLY MAXIMUM THREE BOOKS ARE ALLOWED !!!!!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
    
    def issue_book(self):
        #Status of issue book
        try:
            self.cursor.execute(f"with temp as (select books_data.book_name,books_data.book_id,user_id from books_data inner join user_book_data on user_book_data.book_id = books_data.book_id) select book_name from temp where user_id ='{self.id_}'") 
            cmd = self.cursor.fetchall()
            print()
            if len(cmd)>0:
                for i in range(len(cmd)):
                    print("\n\t\t\t\t\t YOUR ISSUES BOOK IS  :  ",cmd[i][0])
                    print('__'*63)
            else:
                print("\n \t\t\t\t\tUSER HAS NOT ISSUED ANY BOOK!!!!!!!!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!!!!")
        
    def get_book_by_name(self): 
        #if you want to book issue by book name
        try:
            self.cursor.execute(f"select count({self.id_}) from user_book_data where user_id = '{self.id_}'")
            ide = self.cursor.fetchall()
            if ide[0][0]<3:
                book_name = input("\n ENTER THE NAME OF BOOK WHICH YOU WANT  :  ").strip().title()
                self.cursor.execute(f"select book_id,book_name from books_data where book_name = '{book_name}'")
                b = self.cursor.fetchall()

                l = []
                if len(b)>0:
                    self.cursor.execute(f"select book_name from books_data where book_id = '{b[0][0]}'")
                    id1 = self.cursor.fetchall()
                    self.cursor.execute(f"select user_name from user_data where id = '{self.id_}'")
                    name = self.cursor.fetchall()
                    for t in b:
                        l.append(t[1])
                if book_name in l:
                    self.cursor.execute("select book_id from user_book_data")
                    id__ = self.cursor.fetchall()
                    p = [i[0] for i in id__]
                    v = b[0][0]
                    if v not in p:
                        t = (date.today()+timedelta(days=30)).isoformat()
                        cmd = f"insert into user_book_data(user_id,student_name,book_id,book_name,Submission_date) values({self.id_},'{name[0][0]}',{v},'{id1[0][0]}','{t}')"
                        self.cursor.execute(cmd)
                        self.db.commit()
                        print()
                        print("\n\t\t\t\t\t*************** BOOK ISSUED SUCCESSFULLY ***************")
                    else:
                        print("\n\t\t\t\t\t THIS BOOK IS NOT PRESENT IN THE LIBRARY !!!!!!!!!!!!!")
                else:
                    print("\n\t\t\t\t\t THIS BOOK IS NOT PRESENT IN LIBRARY !!!!!!!!!!!!")
            else:
                print("\n\t\t\t\t\t ONLY MAXIMUM THREE BOOKS ARE ALLOWED !!!!!!!!!!!!!!")
       
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")   
            
    def user_issue_book_status(self):
        #User issue books status
        try:
            self.cursor.execute(f"with temp as(select books_data.book_name,user_id,books_data.book_id,submission_date from books_data inner join user_book_data on user_book_data.book_id = books_data.book_id) select user_id,book_name,book_id,submission_date from temp where user_id ='{self.id_}'")
            d = self.cursor.fetchall()
            print()
            if len(d)>0:
                for i in d:
                    print(f"\n USER ID : {i[0]}   &    NAME OF BOOK  : {i[1]}    &     BOOK ID   : {i[2]}   &  SUBMISSION DATE : {i[3]}")
                    print("__"*60)
            else:
                print("\n\t\t\t\t\t USER HAS NOT ISSUEED ANY BOOK!!!!!!!!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
            
    def forgot_password(self):
        #Recover password via smtp
        try:
            from_email = "hiteshnarwani111@gmail.com"
            password = getpass("\n PLEASE ENTER THE PASSWORD OF FROM EMAIL  :  ")
            t = date.today().isoformat()
            message = MIMEMultipart()
            message['From'] = "hiteshnarwani111@gmail.com"
            self.cursor.execute(f"select user_email from user_data where id = '{self.id_}'")                 
            email = self.cursor.fetchall()
            host = "smtp.gmail.com"
            port = 465
            message['To'] = email[0][0]
            message['Subject'] = "Forgot Password"
            l = []
            msg = ""
            for j in range(6):
                s = randint(0,9)
                msg+=str(s)
            plain = MIMEText(msg, 'plain')
            message.attach(plain)


            with smtplib.SMTP_SSL(host, port) as server:
                server.login(from_email, password)
                server.sendmail(from_email, message['To'], message.as_string())

            otp = input("\nPLEASE ENTER THE OTP  :  ")
            if msg == otp:
                while True:
                    password = getpass("\nPLEASE ENTER THE NEW PASSWORD  :  ")
                    count=0
                    upr=0
                    lwr=0
                    sp=0
                    for s in password:
                        if s.isnumeric():
                            count+=1
                    for d in password:
                        if  d.isupper():
                            upr+=1
                    for x in password:
                        if  x.islower():
                            lwr+=1  
                    special = "!@#$%^&*()-+?_=,<>/"
                    for o in password:
                        if o in special:
                            sp+=1
                    if len(password)>=8:
                        if count>=1:
                            if upr>=1:
                                if lwr>=1:
                                    if sp>=1:
                                        cmd = f"update user_data set password = '{password}' where id = '{self.id_}'"         
                                        self.cursor.execute(cmd)
                                        self.db.commit()
                                        print()
                                        print("\n\t\t\t\t\t********** YOUR PASSWORD HAS BEEN CHANGED ************")
                                        print("*"*100)
                                        break
                                    else:
                                        print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                                else:
                                    print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                            else:
                                print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                        else:
                            print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                    else:
                        print("\n\t\t\t\t\t  PLEASE ENTER THE MININUM 8 CHARACTER!!!!!!!!!!!!!!")
            else:
                print("\n\t\t\t\t\t PLEASE ENTER THE VALID OTP!!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
            
            
            
            
    def submit(self):
        #Submit book
        try:
            cmd = f"select * from user_book_data where user_id = {self.id_}"
            self.cursor.execute(cmd)
            d = self.cursor.fetchall()
            if len(d)>0:
                for i in d:
                    print(f"\n\tUSER_ID  :  {i[0]}\t&\tSTUDENT_NAME  :  {i[1]}\t&\tBOOK_ID  :  {i[2]}\t&\tBOOK_NAME  :  {i[3]}\t&\tSUBMISSION_DATE  :  {i[4]}")
                    print("__"*60)
                b_i = int(input("\n PLEASE ENTER THE BOOK ID  :  "))
                r = f"delete from user_book_data where book_id = {b_i}"
                self.cursor.execute(r)
                self.db.commit()
            else:
                print("\n\t\t\t\t\t USER HAS NOT ISSUEED ANY BOOK!!!!!!!!!!!!!!!!!")
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
class Sign_up_1:  
    #Class sign_up with related methods
    db =  sql.connect(host='localhost', port=3306, user='root', password='',database = "library")
    cursor = db.cursor()
    
    def sign_up(self):
        # New registration of user
        try:
            while True:
                name = input("\n PLEASE ENTER THE NAME  :  ").strip().title()
                age = int(input("\n PLEASE ENTER THE AGE  :  "))
                email = input("\n PLEASE ENTER THE EMAIL  :  ")
                regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                if(re.fullmatch(regex, email)):
                    print("\n FOR EMAIL VARIFICATION") 
                    from_email = "hiteshnarwani111@gmail.com"
                    message = MIMEMultipart()
                    message['From'] = "hiteshnarwani111@gmail.com"
                    password = getpass("\n PLEASE ENTER THE PASSWORD OF FROM EMAIL  :  ")
                    host = "smtp.gmail.com"
                    port = 465
                    message['To'] = email
                    message['Subject'] = "VARIFICATION"
                    l = []
                    msg = ""
                    for j in range(6):
                        s = randint(0,9)
                        msg+=str(s)
                    plain = MIMEText(msg, 'plain')
                    message.attach(plain)


                    with smtplib.SMTP_SSL(host, port) as server:
                        server.login(from_email, password)
                        server.sendmail(from_email, message['To'], message.as_string())
                    otp = getpass("\n PLEASE ENTER THE OTP  :  ")
                    if msg == otp:
                        password = getpass("\n PLEASE ENTER THE STRONG PASSWORD  :  ")
                        count=0
                        upr=0
                        lwr=0
                        sp=0
                        for s in password:
                            if s.isnumeric():
                                count+=1
                        for d in password:
                            if  d.isupper():
                                upr+=1
                        for x in password:
                            if  x.islower():
                                lwr+=1  
                        special = "!@#$%^&*()-+?_=,<>/"
                        for o in password:
                            if o in special:
                                sp+=1
                        if len(password)>=8:
                            if count>=1:
                                if upr>=1:
                                    if lwr>=1:
                                        if sp>=1:
                                            cmd = f"insert into user_data(user_name,user_age,user_email,password) values('{name}',{age},'{email}','{password}')"         
                                            self.cursor.execute(cmd)
                                            self.db.commit()
                                            self.cursor.execute(f"select id from user_data")
                                            cmd = self.cursor.fetchall()
                                            print()
                                            print("\n\t\t\t\t\t THANKS FOR SIGNUP")
                                            print(f"\n\t\t\t\t\t YOUR USER ID IS {cmd[-1][0]}")
                                            print("*"*100)
                                            break
                                        else:
                                            print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                                            break
                                    else:
                                        print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                                        break
                                else:
                                    print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                                    break
                            else:
                                print("\n\t\t\t\t\t PLEASE ENTER THE STRONG PASSWORD!!!!!!!!!!!")
                                break
                        else:
                            print("\n\t\t\t\t\t  PLEASE ENTER THE MININUM 8 CHARACTER!!!!!!!!!!!!!!")
                            break
                    else:
                        print("\n\t\t\t\t\t PLEASE ENTER THE VALID OTP !!!!!!!!!!!!!")
                        break
                else:
                    print("\n\t\t\t\t\t PLEASE ENTER A VALID EMAIL !!!!!!!!!!!!!!")
                    break
        
        except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")
              

    
            
def main():
    try:
        from getpass import getpass
        db =  sql.connect(host='localhost', port=3306, user='root', password='',database = "library")
        cursor = db.cursor()
        print("#"*120)
        print(" WELCOME TO THE LIBRARY MANAGEMENT SYSTEM ".rjust(80))
        print("#"*120)
        print()
        while True:
                print("\n\t\t\t\t\t 1. LIBRARIAN LOGIN \n\t\t\t\t\t 2. USER LOGIN \n\t\t\t\t\t 3. SIGN UP \n\t\t\t\t\t 4. EXIT")
                choice = input("\n PLEASE CHOOSE CHOICE FROM THESE ITEMS  :  ").strip().lower()
                if choice == "1":
                    l_id = int(input("\n PLEASE ENTER THE ID  :  "))
                    cursor.execute("select l_id,password from librarian_data")    
                    user_data_name = cursor.fetchall()
                    l = [i[0] for i in user_data_name]
                    if l_id in l:
                        password = getpass("\n PLEASE ENTER THE PASSWORD  :  ")
                        print()
                        t = l.index(l_id)
                        if user_data_name[t][1] == password:
                            ob = librarian(l_id,password)
                            while True:
                                print()
                                print("\n\t\t\t\t\t 1. SHOW DETAILS OF BOOK \n\t\t\t\t\t 2. USERS WHO HAVE ISSUED THE BOOKS \n\t\t\t\t\t 3. USERS WHO HAVE TO SUBMIT THEIR BOOK ON THAT DATE \n\t\t\t\t\t 4. SEND EMAIL\n\t\t\t\t\t 5. INSERT BOOK \n\t\t\t\t\t 6. SUBMIT BOOK\n\t\t\t\t\t 7. LOGOUT")
                                print()
                                print("*"*120)
                                ch = input("\n PLEASE ENTER THE CHOOSE CHOICE FROM THESE  :  ")
                                print()
                                if ch == "1":
                                    ob.show_details_of_book()
                                elif ch == "2":
                                    ob.users_who_have_issued_the_books()
                                elif ch == "3":
                                    ob.users_who_have_to_submit_their_book_on_that_date()
                                elif ch == "4":
                                    ob.send_email()
                                elif ch == "5":
                                    ob.insert_book()
                                elif ch == "6":
                                    ob.submit_book()
                                elif ch == "7":
                                    break
                                else:
                                    print("\n\t\t\t\t\t PLEASE ENTER THE VALID INPUT !!!!!!!!!!!!!!!")        
                        else:
                            print("\n\t\t\t\t\t PLEASE ENTER VALID PASSWORD !!!!!!!!!!!!!!!!")
                    else:
                        print("\n\t\t\t\t\t PLEASE ENTER THE VALID ID !!!!!!!!!!!!!!!")



                elif choice == "2":    
                        from getpass import getpass
                        user_name = int(input("\n PLEASE ENTER THE ID  :  "))
                        cursor.execute("select id,password from user_data")
                        user_data_name = cursor.fetchall()
                        l = [i[0] for i in user_data_name]
                        if user_name in l:
                            password = getpass("\n PLEASE ENTER THE PASSWORD  :  ")

                            t = l.index(user_name)
                            if user_data_name[t][1] == password:
                                id_ = user_data_name[t][0]
                                while True:
                                    print()
                                    print("\n\t\t\t\t\t 1. GET BOOK BY AUTHOR \n\t\t\t\t\t 2. GET BOOK BY CATEGORY \n\t\t\t\t\t 3. ISSUE BOOK \n\t\t\t\t\t 4. GET BOOK BY NAME \n\t\t\t\t\t 5. USER ISSUE BOOK STATUS \n\t\t\t\t\t 6. FORGOT PASSWORD \n\t\t\t\t\t 7. SUBMIT \n\t\t\t\t\t 8. LOGOUT") 
                                    print()
                                    print("*"*120)
                                    op = input("\n PLEASE CHOOSE CHOICE FROM THESE ITEMS  :  ")
                                    obj = user_login(user_name,password,id_)
                                    if op == "1":
                                        obj.get_book_by_author()

                                    elif op == "2":
                                        obj.Get_book_by_category()

                                    elif op == "3":
                                        obj.issue_book()

                                    elif op == "4":
                                        obj.get_book_by_name()

                                    elif op == "5":
                                        obj.user_issue_book_status()
                                    elif op == "6":
                                        obj.forgot_password()
                                    elif op == "7":
                                        obj.submit()
                                    elif op == "8":
                                        break
                                    else:
                                        print("\n\t\t\t\t\t PLEASE ENTER THE VALID INPUT !!!!!!!!!!!!!!!!!!!")
                            else:
                                print("\n\t\t\t\t\t PLEASE ENTER VALID PASSWORD !!!!!!!!!!!!!!!!")
                        else:
                            print("\n\t\t\t\t\t PLEASE ENTER THE VALID USER ID !!!!!!!!!!!!!!!")
                elif choice == "3":
                    obej = Sign_up_1()
                    obej.sign_up()
                elif choice == "4":
                    break

                else:
                    print("\n\t\t\t\t\t PLEASE ENTER THE VALID INPUT !!!!!!!!!!!!!!!!!!!!")
    except Exception as error:
            print(f"\n\t\t\t\t\tPLEASE ENTER THE INPUT IN NUMBER!!!!!!")

main()

########################################################################################################################
                                       WELCOME TO THE LIBRARY MANAGEMENT SYSTEM 
########################################################################################################################


					 1. LIBRARIAN LOGIN 
					 2. USER LOGIN 
					 3. SIGN UP 
					 4. EXIT

 PLEASE CHOOSE CHOICE FROM THESE ITEMS  :  4


In [24]:
import pymysql as sql
class librarian:
    #cursor.execute("use library")
            
            
    def __init__(self,name,password):
        self.name = name
        self.password = password
        self.db =  sql.connect(host='localhost', port=3306, user='root', password='',database = "library")
        self.cursor = self.db.cursor()
        
    def show_details_of_book(self):
        self.cursor.execute('select * from books_data')
        detail = self.cursor.fetchall()
        for i in detail:
            print(f"\n Book_Id  :  {i[0]}   &     Book_Name   :   {i[1]}      &     Book_Author   :   {i[2]}    &     Book_Published_Year   :    {i[3]}      &       Book_Category   :  {i[4]} " )
            print("__"*60)
            print()
            
            
    def users_who_have_to_submit_their_book_on_that_date(self):
        t = date.today().isoformat()
        cursor.execute(f"with temp as (select user_id,user_name,id_of_books,submission_date from user_book_data inner join user_data on user_book_data.user_id = user_data.id) select * from temp where submission_date = '{t}'")
        detail = cursor.fetchall()
        for i in detail:
            print(f"\n User_Id :  {i[0]}  &    User_Name   :   {i[1]}     &     Book_Id   :   {i[2]}    &      Submission_Date    :   {i[3]}")
            print("__"*60)
                       
        
        
            
    def users_who_have_issued_the_books(self):
        cursor.execute("with temp as (select * from user_book_data inner join user_data on user_book_data.user_id = user_data.id) select id,user_name,id_of_books,submission_date from temp;")
        new = cursor.fetchall()
        for i in new:
            print(f" \n USER ID :  {i[0]}   &   USER NAME :  {i[1]}   &    ID OF BOOKS   :   {i[2]}    &     SUBMISSION DATE  :   {i[3]}")
            print("__"*60)
            print()
            
            
        db.close()   
            

In [ ]:
obj = librarian('simran', 'admin')

In [16]:
obj.show_details_of_book()


 Book_Id  :  1001   &     Book_Name   :   Treasure Island      &     Book_Author   :   Robert Louis Stevenson    &     Book_Published_Year   :    1883      &       Book_Category   :  Action 
________________________________________________________________________________________________________________________


 Book_Id  :  1002   &     Book_Name   :   Da Vinci Code      &     Book_Author   :   Dan Brown    &     Book_Published_Year   :    2003      &       Book_Category   :  Action 
________________________________________________________________________________________________________________________


 Book_Id  :  1003   &     Book_Name   :   Catching Fire      &     Book_Author   :   Suzanne Collins    &     Book_Published_Year   :    2009      &       Book_Category   :  Action 
________________________________________________________________________________________________________________________


 Book_Id  :  1004   &     Book_Name   :   The Great Gatsby      &     Book_Author